In [3]:
# unified_ef_pipeline_pytorch.py
import os
import random
import numpy as np
import pandas as pd
import joblib
import math
import time
from tqdm import tqdm

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, classification_report

# plotting (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# signal utils
from scipy.signal.windows import gaussian
from scipy.signal import find_peaks

# reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
OUTPUT_DIR = './output_pytorch/'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --------------------------
# Cardiac Feature Extractor (same logic)
# --------------------------
class CardiacFeatureExtractor:
    def __init__(self, sampling_rate=500):
        self.sampling_rate = sampling_rate

    def detect_r_peaks(self, ecg_signal):
        try:
            height_threshold = np.percentile(ecg_signal, 75)
            distance = int(0.25 * self.sampling_rate)
            peaks, _ = find_peaks(ecg_signal, height=height_threshold, distance=distance)
            return peaks
        except Exception:
            return np.arange(100, len(ecg_signal)-100, int(0.8 * self.sampling_rate))

    def compute_pep_lvet_ratio(self, ecg, scg_signal):
        try:
            r_peaks = self.detect_r_peaks(ecg)
            if len(r_peaks) < 3:
                return 80.0, 300.0, 0.27
            r_times = r_peaks * (1000.0 / self.sampling_rate)
            scg_peaks, _ = find_peaks(scg_signal, distance=int(0.15 * self.sampling_rate))
            scg_times = scg_peaks * (1000.0 / self.sampling_rate)

            pep_values = []
            for r_time in r_times[:5]:
                subsequent_scg = scg_times[scg_times > r_time]
                if len(subsequent_scg) > 0:
                    pep = subsequent_scg[0] - r_time
                    if 30 <= pep <= 150:
                        pep_values.append(pep)
            pep_ms = np.median(pep_values) if pep_values else 80.0

            lvet_values = []
            for i in range(min(5, len(r_peaks)-1)):
                r_start = r_times[i]
                r_end = r_times[i+1]
                scg_in_beat = scg_times[(scg_times > r_start + 50) & (scg_times < r_end - 50)]
                if len(scg_in_beat) >= 2:
                    lvet = scg_in_beat[-1] - scg_in_beat[0]
                    if 200 <= lvet <= 450:
                        lvet_values.append(lvet)
            lvet_ms = np.median(lvet_values) if lvet_values else 300.0

            ratio = pep_ms / lvet_ms if lvet_ms > 0 else 0.27
            return pep_ms, lvet_ms, ratio
        except Exception:
            return 80.0, 300.0, 0.27

# --------------------------
# Synthetic Data Generator (keeps same behavior)
# --------------------------
class SyntheticDataGenerator:
    def __init__(self, n_samples=1000, sequence_length=3000, sampling_rate=500):
        self.n_samples = n_samples
        self.sequence_length = sequence_length
        self.sampling_rate = sampling_rate
        self.feat_ext = CardiacFeatureExtractor(sampling_rate)

    def generate_ecg_signal(self, heart_rate, abnormality_level=0):
        t = np.linspace(0, 6, self.sequence_length)
        ecg = np.zeros_like(t)
        rr_interval = 60.0 / heart_rate
        rr_variability = 0.1 * abnormality_level if abnormality_level > 0 else 0.02
        current_time = 0.0
        while current_time < 6.0:
            idx = int(current_time * self.sampling_rate)
            if idx < len(ecg) - 100:
                qrs_width = int((0.08 + 0.02 * abnormality_level) * self.sampling_rate)
                qrs = gaussian(qrs_width, std=qrs_width/6) * (1.2 - 0.4 * abnormality_level)
                end_idx = min(idx + qrs_width, len(ecg))
                ecg[idx:end_idx] += qrs[:end_idx-idx]

                t_start = idx + int(0.2 * self.sampling_rate)
                t_width = int(0.15 * self.sampling_rate)
                if t_start + t_width < len(ecg):
                    t_wave = gaussian(t_width, std=t_width/4) * (0.3 - 0.15 * abnormality_level)
                    ecg[t_start:t_start+t_width] += t_wave
            current_time += rr_interval * (1 + np.random.uniform(-rr_variability, rr_variability))
        noise_level = 0.02 + 0.03 * abnormality_level
        ecg += np.random.normal(0, noise_level, len(ecg))
        return ecg

    def generate_scg_signal(self, r_peaks, pep_ratio, lvet_ratio):
        scg = np.zeros(self.sequence_length)
        for r_peak in r_peaks:
            if r_peak >= self.sequence_length - 200:
                continue
            pep_delay = int((0.04 + 0.03 * pep_ratio) * self.sampling_rate)
            ao_idx = r_peak + pep_delay
            if ao_idx < self.sequence_length:
                ao_width = int(0.08 * self.sampling_rate)
                ao_wave = gaussian(ao_width, std=8) * (0.6 - 0.3 * pep_ratio)
                end_idx = min(ao_idx + ao_width, self.sequence_length)
                scg[ao_idx:end_idx] += ao_wave[:end_idx-ao_idx]
            lvet_delay = int((0.30 - 0.08 * lvet_ratio) * self.sampling_rate)
            ac_idx = r_peak + lvet_delay
            if ac_idx < self.sequence_length:
                ac_width = int(0.06 * self.sampling_rate)
                ac_wave = gaussian(ac_width, std=6) * (0.4 - 0.2 * lvet_ratio)
                end_idx = min(ac_idx + ac_width, self.sequence_length)
                scg[ac_idx:end_idx] += ac_wave[:end_idx-ac_idx]
        scg += np.random.normal(0, 0.01, len(scg))
        return scg

    def generate_dataset(self, verbose=True):
        X_raw = []
        X_features = []
        y_ef = []
        y_category = []
        summary_data = []

        if verbose:
            print(f"Generating {self.n_samples} samples...")

        for i in range(self.n_samples):
            category_prob = np.random.random()
            if category_prob < 0.33:
                ef = np.random.normal(62, 3)
                category = "Normal"
                abnormality = 0.0
                target_pep_ratio = np.random.uniform(0.20, 0.30)
            elif category_prob < 0.66:
                ef = np.random.normal(45, 3)
                category = "Mildly Reduced"
                abnormality = 0.5
                target_pep_ratio = np.random.uniform(0.30, 0.40)
            else:
                ef = np.random.normal(32, 5)
                category = "Abnormal"
                abnormality = 1.0
                target_pep_ratio = np.random.uniform(0.40, 0.55)

            ef = max(20, min(80, ef))
            if category == "Normal":
                hr = np.random.normal(70, 8)
            elif category == "Mildly Reduced":
                hr = np.random.normal(75, 10)
            else:
                hr = np.random.normal(85, 12)
            hr = max(50, min(120, hr))

            ecg = self.generate_ecg_signal(hr, abnormality)
            r_peaks = self.feat_ext.detect_r_peaks(ecg)
            target_lvet_ratio = 1.0 - target_pep_ratio
            scg = self.generate_scg_signal(r_peaks, target_pep_ratio, target_lvet_ratio)

            t = np.arange(self.sequence_length) / self.sampling_rate
            hr_hz = hr / 60.0
            accel_x = np.random.normal(0, 0.03, self.sequence_length) + 0.02 * np.sin(2 * np.pi * hr_hz * t)
            accel_y = np.random.normal(0, 0.03, self.sequence_length) + 0.015 * np.sin(2 * np.pi * hr_hz * t)
            accel_z = np.random.normal(0, 0.03, self.sequence_length) + 0.025 * np.sin(2 * np.pi * hr_hz * t)

            accel_x += scg * 0.1
            accel_y += scg * 0.08
            accel_z += scg * 0.12

            audio = np.random.normal(0, 0.01, self.sequence_length)
            for r_peak in r_peaks[:10]:
                if r_peak + 100 < len(audio):
                    s1_idx = r_peak + int(0.02 * self.sampling_rate)
                    s1_dur = int(0.04 * self.sampling_rate)
                    if s1_idx + s1_dur < len(audio):
                        audio[s1_idx:s1_idx+s1_dur] += gaussian(s1_dur, std=4) * 0.15
                    s2_idx = r_peak + int(0.35 * self.sampling_rate)
                    s2_dur = int(0.03 * self.sampling_rate)
                    s2_strength = 0.12 - 0.08 * abnormality
                    if s2_idx + s2_dur < len(audio):
                        audio[s2_idx:s2_idx+s2_dur] += gaussian(s2_dur, std=3) * s2_strength

            pep_ms, lvet_ms, pep_lvet_ratio = self.feat_ext.compute_pep_lvet_ratio(ecg, scg)

            multi_ch = np.stack([ecg, accel_x, accel_y, accel_z, audio], axis=1).astype(np.float32)
            X_raw.append(multi_ch)

            features = {
                'heart_rate': hr,
                'pep_ms': pep_ms,
                'lvet_ms': lvet_ms,
                'pep_lvet_ratio': pep_lvet_ratio,
                'ecg_mean': float(np.mean(ecg)),
                'ecg_std': float(np.std(ecg)),
                'scg_mean': float(np.mean(scg)),
                'scg_std': float(np.std(scg)),
            }
            X_features.append(features)
            y_ef.append(ef)
            y_category.append(category)
            summary_data.append({
                'sample_id': i,
                'ejection_fraction': ef,
                'category': category,
                'heart_rate': hr,
                'pep_ms': pep_ms,
                'lvet_ms': lvet_ms,
                'pep_lvet_ratio': pep_lvet_ratio,
                'abnormality_level': abnormality
            })

            if (i + 1) % 200 == 0 and verbose:
                print(f"Generated {i + 1}/{self.n_samples} samples")

        X_raw = np.array(X_raw)
        X_df = pd.DataFrame(X_features)
        y_ef = np.array(y_ef)
        y_category = np.array(y_category)
        summary_df = pd.DataFrame(summary_data)
        return X_raw, X_df, y_ef, y_category, summary_df

# --------------------------
# PyTorch Dataset
# --------------------------
class CardiacDataset(Dataset):
    def __init__(self, X, y):
        # X: numpy (N, seq_len, n_channels)
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]  # (seq_len, n_channels)
        # switch to (channels, seq_len) for conv1d
        x = np.transpose(x, (1, 0)).astype(np.float32)
        y = self.y[idx]
        return torch.from_numpy(x), torch.tensor(y, dtype=torch.float32)

# --------------------------
# Model (mirrors Keras architecture)
# --------------------------
class EFNet(nn.Module):
    def __init__(self, in_channels):
        super(EFNet, self).__init__()
        # Conv1d expects (batch, channels, seq_len)
        self.conv1 = nn.Conv1d(in_channels, 32, kernel_size=9, padding=4)
        self.bn1 = nn.BatchNorm1d(32)
        self.pool1 = nn.MaxPool1d(4)

        self.conv2 = nn.Conv1d(32, 64, kernel_size=7, padding=3)
        self.bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.MaxPool1d(4)

        self.conv3 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.bn3 = nn.BatchNorm1d(128)
        self.pool3 = nn.MaxPool1d(4)

        self.global_pool = nn.AdaptiveAvgPool1d(1)  # like GlobalAveragePooling1D
        self.drop1 = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128, 64)
        self.drop2 = nn.Dropout(0.2)
        self.out = nn.Linear(64, 1)

    def forward(self, x):
        # x: (batch, channels, seq_len)
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = self.global_pool(x)  # (batch, 128, 1)
        x = x.view(x.size(0), -1)  # (batch, 128)
        x = self.drop1(x)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        x = self.out(x)
        return x.squeeze(1)

# --------------------------
# Trainer (simple training loop with early stopping)
# --------------------------
class Trainer:
    def __init__(self, model, device=DEVICE):
        self.model = model.to(device)
        self.device = device

    def fit(self, train_loader, val_loader, epochs=50, lr=1e-3, checkpoint_path=None, patience=10):
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        criterion = nn.MSELoss()
        best_val = math.inf
        best_epoch = 0
        history = {'train_loss': [], 'val_loss': []}

        for epoch in range(1, epochs+1):
            self.model.train()
            train_losses = []
            for xb, yb in train_loader:
                xb = xb.to(self.device)
                yb = yb.to(self.device)
                preds = self.model(xb)
                loss = criterion(preds, yb)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                train_losses.append(loss.item())

            val_losses = []
            self.model.eval()
            with torch.no_grad():
                for xb, yb in val_loader:
                    xb = xb.to(self.device)
                    yb = yb.to(self.device)
                    preds = self.model(xb)
                    loss = criterion(preds, yb)
                    val_losses.append(loss.item())

            avg_train = np.mean(train_losses)
            avg_val = np.mean(val_losses)
            history['train_loss'].append(avg_train)
            history['val_loss'].append(avg_val)

            print(f"Epoch {epoch}/{epochs} - train_loss: {avg_train:.4f} - val_loss: {avg_val:.4f}")

            if avg_val < best_val - 1e-6:
                best_val = avg_val
                best_epoch = epoch
                if checkpoint_path:
                    torch.save(self.model.state_dict(), checkpoint_path)
                print(f"  New best val {best_val:.5f} saved.")
            if epoch - best_epoch >= patience:
                print(f"Early stopping at epoch {epoch}. Best epoch was {best_epoch}.")
                break
        # load best
        if checkpoint_path and os.path.exists(checkpoint_path):
            self.model.load_state_dict(torch.load(checkpoint_path, map_location=self.device))
        return history

# --------------------------
# Helper: EF -> category
# --------------------------
def ef_to_category(ef):
    if ef >= 50:
        return "Normal"
    elif ef >= 41:
        return "Mildly Reduced"
    else:
        return "Abnormal"

# --------------------------
# Inference function (user requested)
# --------------------------
def infer_ef(model_path, scalers_path, ef_scaler_path, sample, device=None):
    """
    Robust inference helper.

    Args:
        model_path: path to saved model state_dict (.pth/.pt)
        scalers_path: joblib file containing dict of per-channel StandardScaler objects
        ef_scaler_path: joblib file for the EF target scaler
        sample: numpy array of shape
                - (seq_len, n_channels)  OR
                - (n_channels, seq_len)  OR
                - (batch, seq_len, n_channels)
        device: torch device string or None (auto)
    Returns:
        dict with keys:
            'ef_pred': float or list of floats
            'category': str or list of strs
    """
    # choose device
    device = DEVICE if device is None else torch.device(device)

    # load scalers
    scalers = joblib.load(scalers_path)    # dict like {'channel_0': scaler, ...}
    ef_scaler = joblib.load(ef_scaler_path)

    # determine expected n_channels
    n_channels_expected = len(scalers)

    arr = np.array(sample, dtype=np.float32)

    # Normalize shape handling:
    # Accept shapes: (seq_len, n_channels), (n_channels, seq_len), (batch, seq_len, n_channels)
    if arr.ndim == 2:
        # ambiguous: either (seq_len, n_channels) or (n_channels, seq_len)
        if arr.shape[1] == n_channels_expected:
            # (seq_len, n_channels) -> good
            arr = arr[np.newaxis, ...]  # make batch
        elif arr.shape[0] == n_channels_expected:
            # (n_channels, seq_len) -> transpose to (seq_len, n_channels)
            arr = arr.T[np.newaxis, ...]
        else:
            raise ValueError(f"Cannot infer channels. Got shape {arr.shape} but scalers expect {n_channels_expected} channels.")
    elif arr.ndim == 3:
        # assume (batch, seq_len, n_channels) or (batch, n_channels, seq_len)
        if arr.shape[2] == n_channels_expected:
            # good: (batch, seq_len, n_channels)
            pass
        elif arr.shape[1] == n_channels_expected:
            # (batch, n_channels, seq_len) -> transpose each sample
            arr = np.transpose(arr, (0, 2, 1))
        else:
            raise ValueError(f"Cannot infer channels from batch. Got shape {arr.shape} but scalers expect {n_channels_expected} channels.")
    else:
        raise ValueError("sample must be 2D or 3D numpy array")

    # Now arr is (batch, seq_len, n_channels)
    batch_size, seq_len, n_channels = arr.shape

    # Apply per-channel scalers (they were fit on flattened values)
    X_scaled = np.zeros_like(arr)
    for ch in range(n_channels):
        scaler = scalers[f'channel_{ch}']
        flat = arr[:, :, ch].reshape(-1, 1)
        transformed = scaler.transform(flat).reshape(batch_size, seq_len)
        X_scaled[:, :, ch] = transformed

    # Convert to (batch, channels, seq_len) for Conv1d
    X_tensor = torch.from_numpy(np.transpose(X_scaled, (0, 2, 1))).to(device).float()

    # Build model architecture matching training and load state_dict
    in_channels = X_tensor.shape[1]
    model = EFNet(in_channels).to(device)
    state = torch.load(model_path, map_location=device)

    # If user saved a full checkpoint dict vs state_dict, handle both:
    if isinstance(state, dict) and ('model_state_dict' in state or 'state_dict' in state):
        # common patterns: {'model_state_dict':..., ...} or {'state_dict':...}
        key = 'model_state_dict' if 'model_state_dict' in state else 'state_dict'
        model.load_state_dict(state[key])
    else:
        # assume raw state_dict
        model.load_state_dict(state)

    model.eval()
    with torch.no_grad():
        preds_scaled = model(X_tensor).cpu().numpy().reshape(-1, 1)

    # inverse transform using ef_scaler if possible
    try:
        preds_unscaled = ef_scaler.inverse_transform(preds_scaled).flatten()
    except Exception:
        preds_unscaled = preds_scaled.flatten()

    # return single or batch friendly
    if len(preds_unscaled) == 1:
        val = float(preds_unscaled[0])
        return {'ef_pred': val, 'category': ef_to_category(val)}
    else:
        vals = preds_unscaled.tolist()
        cats = [ef_to_category(v) for v in vals]
        return {'ef_pred': vals, 'category': cats}


# --------------------------
# Full main (generate -> train -> eval)
# --------------------------
def main():
    print("=== CARDIAC EF PREDICTION PIPELINE (PyTorch) ===")
    # Config
    n_samples = 1000
    seq_len = 3000
    sampling_rate = 500
    batch_size = 16
    epochs = 50
    patience = 8
    lr = 1e-3

    # Generate
    generator = SyntheticDataGenerator(n_samples=n_samples, sequence_length=seq_len, sampling_rate=sampling_rate)
    X_raw, X_df, y_ef, y_category, summary_df = generator.generate_dataset(verbose=True)

    print(f"Generated dataset shapes: X_raw {X_raw.shape}, y_ef {y_ef.shape}")
    summary_df.to_csv(os.path.join(OUTPUT_DIR, 'cardiac_dataset_summary.csv'), index=False)

    # map categories for stratify (just like original)
    category_map = {"Normal": 0, "Mildly Reduced": 1, "Abnormal": 2}
    y_cat_num = np.array([category_map[c] for c in y_category])

    # split
    X_train, X_test, y_train, y_test, y_cat_train, y_cat_test = train_test_split(
        X_raw, y_ef, y_cat_num, test_size=0.2, random_state=SEED, stratify=y_cat_num
    )

    # scale per-channel
    n_channels = X_train.shape[2]
    scalers = {}
    for ch in range(n_channels):
        scaler = StandardScaler()
        X_train_flat = X_train[:, :, ch].reshape(-1, 1)
        X_test_flat = X_test[:, :, ch].reshape(-1, 1)
        scaler.fit(X_train_flat)
        # transform and assign back
        X_train[:, :, ch] = scaler.transform(X_train_flat).reshape(X_train.shape[0], -1)
        X_test[:, :, ch] = scaler.transform(X_test_flat).reshape(X_test.shape[0], -1)
        scalers[f'channel_{ch}'] = scaler

    # scale EF target
    ef_scaler = StandardScaler()
    y_train_scaled = ef_scaler.fit_transform(y_train.reshape(-1, 1)).flatten()
    y_test_scaled = ef_scaler.transform(y_test.reshape(-1, 1)).flatten()

    # Save scalers
    scalers_path = os.path.join(OUTPUT_DIR, 'channel_scalers.joblib')
    ef_scaler_path = os.path.join(OUTPUT_DIR, 'ef_scaler.joblib')
    joblib.dump(scalers, scalers_path)
    joblib.dump(ef_scaler, ef_scaler_path)
    print(f"Saved scalers to {scalers_path} and {ef_scaler_path}")

    # Datasets & loaders (we'll use scaled targets)
    train_ds = CardiacDataset(X_train, y_train_scaled)
    val_ds = CardiacDataset(X_test, y_test_scaled)  # using test as val for simplicity
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0)

    # model
    model = EFNet(in_channels=n_channels)
    trainer = Trainer(model, device=DEVICE)

    checkpoint_path = os.path.join(OUTPUT_DIR, 'best_model.pth')
    history = trainer.fit(train_loader, val_loader, epochs=epochs, lr=lr, checkpoint_path=checkpoint_path, patience=patience)

    # Evaluate on test (unscale predictions)
    # load best model
    model.load_state_dict(torch.load(checkpoint_path, map_location=DEVICE))
    model.to(DEVICE)
    model.eval()
    preds_scaled = []
    y_true = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(DEVICE)
            out = model(xb).cpu().numpy()
            preds_scaled.extend(out.tolist())
            y_true.extend(yb.cpu().numpy().tolist())

    preds_scaled = np.array(preds_scaled).reshape(-1, 1)
    preds = ef_scaler.inverse_transform(preds_scaled).flatten()
    y_true_unscaled = ef_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).flatten()

    # metrics
    mae = mean_absolute_error(y_true_unscaled, preds)
    rmse = math.sqrt(mean_squared_error(y_true_unscaled, preds))
    r2 = r2_score(y_true_unscaled, preds)
    print(f"MAE: {mae:.3f}, RMSE: {rmse:.3f}, R2: {r2:.3f}")

    # categories
    y_true_cat_labels = [ef_to_category(v) for v in y_true_unscaled]
    y_pred_cat_labels = [ef_to_category(v) for v in preds]

    print("Classification report:")
    print(classification_report(y_true_cat_labels, y_pred_cat_labels, target_names=["Normal", "Mildly Reduced", "Abnormal"]))

    # Save final model (state_dict)
    final_model_path = os.path.join(OUTPUT_DIR, 'final_cardiac_ef_model.pth')
    torch.save(model.state_dict(), final_model_path)
    print(f"Saved final model to {final_model_path}")

    # Save results CSV
    results_df = pd.DataFrame({
        'metric': ['MAE', 'RMSE', 'R2'],
        'value': [mae, rmse, r2]
    })
    results_df.to_csv(os.path.join(OUTPUT_DIR, 'final_results.csv'), index=False)
    print(f"Saved results to {os.path.join(OUTPUT_DIR, 'final_results.csv')}")

    # Example inference usage
    example_sample = X_test[0]  # NOTE: already scaled -- infer_ef expects raw sample (unscaled)
    # But we saved scalers and final model, so pass raw (unscaled) sample from original X_raw to infer_ef
    raw_sample = X_raw[0]  # (seq_len, n_channels), raw unscaled
    infer_res = infer_ef(final_model_path, scalers_path, ef_scaler_path, raw_sample, device=str(DEVICE))
    print("Example inference result:", infer_res)

if __name__ == "__main__":
    main()


=== CARDIAC EF PREDICTION PIPELINE (PyTorch) ===
Generating 1000 samples...
Generated 200/1000 samples
Generated 400/1000 samples
Generated 600/1000 samples
Generated 800/1000 samples
Generated 1000/1000 samples
Generated dataset shapes: X_raw (1000, 3000, 5), y_ef (1000,)
Saved scalers to ./output_pytorch/channel_scalers.joblib and ./output_pytorch/ef_scaler.joblib
Epoch 1/50 - train_loss: 0.5317 - val_loss: 0.6814
  New best val 0.68139 saved.
Epoch 2/50 - train_loss: 0.2267 - val_loss: 0.4951
  New best val 0.49515 saved.
Epoch 3/50 - train_loss: 0.1871 - val_loss: 0.1510
  New best val 0.15104 saved.
Epoch 4/50 - train_loss: 0.1902 - val_loss: 0.1679
Epoch 5/50 - train_loss: 0.1788 - val_loss: 0.1029
  New best val 0.10288 saved.
Epoch 6/50 - train_loss: 0.1699 - val_loss: 0.4949
Epoch 7/50 - train_loss: 0.1736 - val_loss: 0.1353
Epoch 8/50 - train_loss: 0.1894 - val_loss: 0.0940
  New best val 0.09402 saved.
Epoch 9/50 - train_loss: 0.1315 - val_loss: 0.1630
Epoch 10/50 - train_lo